> **Jupyter slideshow:** This notebook can be displayed as slides. To view it as a slideshow in your browser, run the following cell:

> `> jupyter nbconvert [this_notebook.ipynb] --to slides --post serve`
 
> To toggle off the slideshow cell formatting, click the `CellToolbar` button, then `View > Cell Toolbar > None`.

<img src="../images/shell-logo.png" width="50" height="50" align="right"/>

<img src="../images/di.png" width="50" height="50" align="right"/>

# Feature Engineering

In [12]:
# ! jupyter nbconvert Python_intro.ipynb --to slides --post serve

## Learning Objectives
*In this lesson, we will go over the following:*

- Introduction to Feature Engineering.
- Application of Feature Engineering
- Feature Engineering with Categorical Columns.
- Feature Engineering with Numerical Columns.
- Feaure Engineering with Business and Domain Understanding.

## Introduction: Manual Feature Engineering

In this notebook, we will explore making features by hand for a Dataset.

- Manual feature engineering can be a tedious process (which is why we use automated feature engineering with featuretools!) and often relies on domain expertise. 

- Since we are limited to domain knowledge of diffirent fields and what makes a person likely to default, we will instead concentrate of getting as much info as possible into the final training dataframe. 

- The idea is that the model will then pick up on which features are important rather than us having to decide that. Basically, our approach is to make as many features as possible and then give them all to the model to use! Later, we can perform feature reduction using the feature importances from the model or other techniques such as PCA. 

In [13]:
# pandas and numpy for data manipulation
import pandas as pd
import numpy as np

# matplotlib and seaborn for plotting
import plotly.express as px
import matplotlib.pyplot as plt
import seaborn as sns

# Suppress warnings from pandas
import warnings
warnings.filterwarnings('ignore')

dataset.json